# Requisitos previos
<hr>

In [87]:
# --------------------------- REQUISITOS PREVIOS ---------------------------

# Pandas, openbabel y plip (quitar el # de las siguientes filas en caso de no tener instalados)
#!conda install -q -y -c anaconda pandas
#!conda install -q -y -c conda-forge openbabel
#!conda install -y -q -c conda-forge plip

# Separar ligandos inhibidores
<hr>

In [ ]:
import os
import glob
import pandas as pd

# Buscar archivos .pdbqt
pdbqts = glob.glob("docking/inhibidores/*.pdbqt")

# Crear carpeta de salida
carpeta_salida = "Mejores_poses/inhibidores"
os.makedirs(carpeta_salida, exist_ok=True)

# Listas para el DataFrame
nombres = []
proteinas = []
rutas = []
afinidades = []
CNNs = []
CNN_afinidades = []

# Procesar cada archivo
for ruta in pdbqts:
    # Extraer nombre y proteína
    nombre_archivo = os.path.basename(ruta)
    molecula = nombre_archivo.split("_L_")[0]
    target = nombre_archivo.split("_L_")[1].split(".")[0]
    
    # Leer el archivo
    with open(ruta, 'r') as f:
        lines = f.readlines()

    # Extraer solo la primera pose (hasta ENDMDL)
    new_lines = []
    for line in lines:
        new_lines.append(line)
        if line.strip() == "ENDMDL":
            break
    
    # Guardar archivo con solo la primera pose
    ruta_salida = os.path.join(carpeta_salida, f"{molecula}_{target}.pdbqt")
    with open(ruta_salida, 'w') as f_out:
        for line in new_lines:
            f_out.write(line)
    
    # Extraer valores de afinidad y CNN
    try:
        afinidades.append(new_lines[1].split()[2])
        CNNs.append(new_lines[2].split()[2])
        CNN_afinidades.append(new_lines[3].split()[2])
    except IndexError:
        afinidades.append(None)
        CNNs.append(None)
        CNN_afinidades.append(None)

    # Guardar datos para el DataFrame
    nombres.append(molecula)
    proteinas.append(target)
    rutas.append(ruta_salida)  # ← CAMBIO AQUÍ

# Crear DataFrame final
df = pd.DataFrame({
    "Molecula": nombres,
    "Target": proteinas,
    "Ruta": rutas,
    "Afinidad": afinidades,
    "CNN": CNNs,
    "CNN afinity": CNN_afinidades
})

In [89]:
df.isna().sum()

Molecula       0
Target         0
Ruta           0
Afinidad       0
CNN            0
CNN afinity    0
dtype: int64

In [90]:
df

,Molecula,Target,Ruta,Afinidad,CNN,CNN afinity
0,EGFR_P_MAPK1_100,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_100_MAP...,-8.56243992,0.492553771,7.70315933
1,EGFR_P_MAPK1_101,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_101_MAP...,-6.49379683,0.3287386,5.92366934
2,EGFR_P_MAPK1_102,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_102_MAP...,-11.8226624,0.179012626,7.56793976
3,EGFR_P_MAPK1_103,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_103_MAP...,-11.0488014,0.812796712,7.96756744
4,EGFR_P_MAPK1_87,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_87_MAPK...,-8.14125919,0.135773346,5.24745607
...,...,...,...,...,...,...
174,SRC_P_MAPK1_124,MAPK1,Mejores_poses/inhibidores/SRC_P_MAPK1_124_MAPK...,-9.97343063,0.870204628,8.17769718
175,SRC_P_MAPK1_125,MAPK1,Mejores_poses/inhibidores/SRC_P_MAPK1_125_MAPK...,-9.02297783,0.659791827,7.74494553
176,SRC_P_STAT3_172,STAT3,Mejores_poses/inhibidores/SRC_P_STAT3_172_STAT...,-7.20385313,0.62575376,6.53311205
177,SRC_P_STAT3_173,STAT3,Mejores_poses/inhibidores/SRC_P_STAT3_173_STAT...,-6.46269941,0.63044697,6.53854179


In [91]:
df["Target"].value_counts()

Target
EGFR     87
MAPK1    43
STAT3    32
SRC      17
Name: count, dtype: int64

# Determinar multitargets inhibidores
<hr>

In [105]:
import pandas as pd

# 1. Filtrar moléculas multitarget
moleculas_multitarget = df.groupby('Molecula')['Target'].nunique()
moleculas_multitarget = moleculas_multitarget[moleculas_multitarget > 1].index

df_multi = df[df['Molecula'].isin(moleculas_multitarget)].copy()

# 2. Agregar un índice para cada target por molécula
df_multi['Target_n'] = df_multi.groupby('Molecula').cumcount() + 1

# 3. Pivotear todas las columnas relevantes, incluyendo Ruta
pivoted = df_multi.pivot(index='Molecula', columns='Target_n', values=['Target', 'Afinidad', 'CNN', 'CNN afinity', 'Ruta'])

# 4. Aplanar el MultiIndex de columnas
pivoted.columns = [f"{col}_{i}" for col, i in pivoted.columns]

# 5. Restablecer el índice para tener "Molecula" como columna normal
resultado = pivoted.reset_index()

# 6. Reordenar columnas si lo deseas (opcional)
cols = ['Molecula'] + sorted([col for col in resultado.columns if col != 'Molecula'])
resultado = resultado[cols]


In [106]:
resultado

,Molecula,Afinidad_1,Afinidad_2,CNN afinity_1,CNN afinity_2,CNN_1,CNN_2,Ruta_1,Ruta_2,Target_1,Target_2
0,EGFR_P_MAPK1_SRC_127,-11.0269213,-10.4229012,7.97076178,8.02541447,0.817015767,0.953700662,Mejores_poses/inhibidores/EGFR_P_MAPK1_SRC_127...,Mejores_poses/inhibidores/EGFR_P_MAPK1_SRC_127...,MAPK1,SRC
1,SRC_P_EGFR_MAPK1_83,-8.36948109,-9.88528061,6.46306086,8.05637741,0.364691794,0.415053397,Mejores_poses/inhibidores/SRC_P_EGFR_MAPK1_83_...,Mejores_poses/inhibidores/SRC_P_EGFR_MAPK1_83_...,EGFR,MAPK1
2,SRC_P_EGFR_MAPK1_84,-9.374156,-10.1891298,6.04878473,7.79595995,0.171025157,0.236191496,Mejores_poses/inhibidores/SRC_P_EGFR_MAPK1_84_...,Mejores_poses/inhibidores/SRC_P_EGFR_MAPK1_84_...,EGFR,MAPK1
3,SRC_P_EGFR_STAT3_85,-10.0429544,-7.37397528,6.90154219,6.87167931,0.303968161,0.756590247,Mejores_poses/inhibidores/SRC_P_EGFR_STAT3_85_...,Mejores_poses/inhibidores/SRC_P_EGFR_STAT3_85_...,EGFR,STAT3
4,SRC_P_EGFR_STAT3_86,-8.16975498,-6.41622734,7.52952576,7.18370819,0.386751026,0.78815496,Mejores_poses/inhibidores/SRC_P_EGFR_STAT3_86_...,Mejores_poses/inhibidores/SRC_P_EGFR_STAT3_86_...,EGFR,STAT3


In [108]:
# 1. Obtener lista de moléculas ya incluidas en resultado
moleculas_a_eliminar = resultado['Molecula'].unique()

# 2. Filtrar df para excluir esas moléculas
df_filtrado = df[~df['Molecula'].isin(moleculas_a_eliminar)].copy()


In [109]:
df_filtrado

,Molecula,Target,Ruta,Afinidad,CNN,CNN afinity
0,EGFR_P_MAPK1_100,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_100_MAP...,-8.56243992,0.492553771,7.70315933
1,EGFR_P_MAPK1_101,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_101_MAP...,-6.49379683,0.3287386,5.92366934
2,EGFR_P_MAPK1_102,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_102_MAP...,-11.8226624,0.179012626,7.56793976
3,EGFR_P_MAPK1_103,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_103_MAP...,-11.0488014,0.812796712,7.96756744
4,EGFR_P_MAPK1_87,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_87_MAPK...,-8.14125919,0.135773346,5.24745607
...,...,...,...,...,...,...
174,SRC_P_MAPK1_124,MAPK1,Mejores_poses/inhibidores/SRC_P_MAPK1_124_MAPK...,-9.97343063,0.870204628,8.17769718
175,SRC_P_MAPK1_125,MAPK1,Mejores_poses/inhibidores/SRC_P_MAPK1_125_MAPK...,-9.02297783,0.659791827,7.74494553
176,SRC_P_STAT3_172,STAT3,Mejores_poses/inhibidores/SRC_P_STAT3_172_STAT...,-7.20385313,0.62575376,6.53311205
177,SRC_P_STAT3_173,STAT3,Mejores_poses/inhibidores/SRC_P_STAT3_173_STAT...,-6.46269941,0.63044697,6.53854179


# Guardar datos de compuestos
<hr>

In [110]:
df_filtrado.to_csv("Resultados_docking/Inhibidores/Duales.csv")

In [111]:
resultado.to_csv("Resultados_docking/Inhibidores/Triples.csv")

# Separar ligandos de las bases
<hr>

In [112]:
import os
import glob
import pandas as pd

# Buscar archivos .pdbqt
pdbqts = glob.glob("docking/bases_conjuntas/*.pdbqt")

# Crear carpeta de salida
carpeta_salida = "Mejores_poses/bases_conjuntas"
os.makedirs(carpeta_salida, exist_ok=True)

# Listas para el DataFrame
nombres = []
proteinas = []
rutas = []
afinidades = []
CNNs = []
CNN_afinidades = []

# Procesar cada archivo
for ruta in pdbqts:
    # Extraer nombre y proteína
    nombre_archivo = os.path.basename(ruta)
    molecula = nombre_archivo.split("_L_")[0]
    target = nombre_archivo.split("_L_")[1].split(".")[0]
    
    # Leer el archivo
    with open(ruta, 'r') as f:
        lines = f.readlines()

    # Extraer solo la primera pose (hasta ENDMDL)
    new_lines = []
    for line in lines:
        new_lines.append(line)
        if line.strip() == "ENDMDL":
            break
    
    # Guardar archivo con solo la primera pose
    ruta_salida = os.path.join(carpeta_salida, f"{molecula}_{target}.pdbqt")
    with open(ruta_salida, 'w') as f_out:
        for line in new_lines:
            f_out.write(line)
    
    # Extraer valores de afinidad y CNN
    try:
        afinidades.append(new_lines[1].split()[2])
        CNNs.append(new_lines[2].split()[2])
        CNN_afinidades.append(new_lines[3].split()[2])
    except IndexError:
        afinidades.append(None)
        CNNs.append(None)
        CNN_afinidades.append(None)

    # Guardar datos para el DataFrame
    nombres.append(molecula)
    proteinas.append(target)
    rutas.append(ruta_salida)  # ← CAMBIO AQUÍ

# Crear DataFrame final
df = pd.DataFrame({
    "Molecula": nombres,
    "Target": proteinas,
    "Ruta": rutas,
    "Afinidad": afinidades,
    "CNN": CNNs,
    "CNN afinity": CNN_afinidades
})


In [113]:
df

,Molecula,Target,Ruta,Afinidad,CNN,CNN afinity
0,EGFR_MAPK1_0,EGFR,Mejores_poses/bases_conjuntas/EGFR_MAPK1_0_EGF...,-8.37202835,0.675964773,7.38650894
1,EGFR_MAPK1_0,MAPK1,Mejores_poses/bases_conjuntas/EGFR_MAPK1_0_MAP...,-6.98167467,0.619238734,7.09848166
2,EGFR_MAPK1_1000,EGFR,Mejores_poses/bases_conjuntas/EGFR_MAPK1_1000_...,-7.45665789,0.912955165,6.42922449
3,EGFR_MAPK1_1000,MAPK1,Mejores_poses/bases_conjuntas/EGFR_MAPK1_1000_...,-7.29448223,0.893933535,6.13602304
4,EGFR_MAPK1_1001,EGFR,Mejores_poses/bases_conjuntas/EGFR_MAPK1_1001_...,-7.58375931,0.771876097,7.05828333
...,...,...,...,...,...,...
2253,EGFR_SRC_1148,SRC,Mejores_poses/bases_conjuntas/EGFR_SRC_1148_SR...,-5.79782248,0.764384985,6.74391508
2254,EGFR_SRC_1149,EGFR,Mejores_poses/bases_conjuntas/EGFR_SRC_1149_EG...,-8.33077812,0.469591349,7.15295744
2255,EGFR_SRC_1149,SRC,Mejores_poses/bases_conjuntas/EGFR_SRC_1149_SR...,-8.07119846,0.50486213,7.44163275
2256,EGFR_STAT3_1150,EGFR,Mejores_poses/bases_conjuntas/EGFR_STAT3_1150_...,-8.68296909,0.702032328,7.67828751


In [114]:
df["Target"].value_counts()

Target
EGFR     1129
MAPK1    1125
SRC         3
STAT3       1
Name: count, dtype: int64

In [115]:
import pandas as pd

# 1. Filtrar moléculas multitarget
moleculas_multitarget = df.groupby('Molecula')['Target'].nunique()
moleculas_multitarget = moleculas_multitarget[moleculas_multitarget > 1].index

df_multi = df[df['Molecula'].isin(moleculas_multitarget)].copy()

# 2. Agregar un índice para cada target por molécula
df_multi['Target_n'] = df_multi.groupby('Molecula').cumcount() + 1

# 3. Pivotear todas las columnas relevantes, incluyendo Ruta
pivoted = df_multi.pivot(index='Molecula', columns='Target_n', values=['Target', 'Afinidad', 'CNN', 'CNN afinity', 'Ruta'])

# 4. Aplanar el MultiIndex de columnas
pivoted.columns = [f"{col}_{i}" for col, i in pivoted.columns]

# 5. Restablecer el índice para tener "Molecula" como columna normal
resultado = pivoted.reset_index()

# 6. Reordenar columnas si lo deseas (opcional)
cols = ['Molecula'] + sorted([col for col in resultado.columns if col != 'Molecula'])
resultado = resultado[cols]


In [116]:
resultado

,Molecula,Afinidad_1,Afinidad_2,CNN afinity_1,CNN afinity_2,CNN_1,CNN_2,Ruta_1,Ruta_2,Target_1,Target_2
0,EGFR_MAPK1_0,-8.37202835,-6.98167467,7.38650894,7.09848166,0.675964773,0.619238734,Mejores_poses/bases_conjuntas/EGFR_MAPK1_0_EGF...,Mejores_poses/bases_conjuntas/EGFR_MAPK1_0_MAP...,EGFR,MAPK1
1,EGFR_MAPK1_1,-8.20100594,-7.20984697,7.36116982,6.69976425,0.366164237,0.307256877,Mejores_poses/bases_conjuntas/EGFR_MAPK1_1_EGF...,Mejores_poses/bases_conjuntas/EGFR_MAPK1_1_MAP...,EGFR,MAPK1
2,EGFR_MAPK1_10,-7.10690355,-7.4140954,6.16407394,6.51218224,0.775842547,0.81885767,Mejores_poses/bases_conjuntas/EGFR_MAPK1_10_EG...,Mejores_poses/bases_conjuntas/EGFR_MAPK1_10_MA...,EGFR,MAPK1
3,EGFR_MAPK1_100,-8.77627087,-9.01747322,8.11514187,7.81839132,0.76803869,0.708038747,Mejores_poses/bases_conjuntas/EGFR_MAPK1_100_E...,Mejores_poses/bases_conjuntas/EGFR_MAPK1_100_M...,EGFR,MAPK1
4,EGFR_MAPK1_1000,-7.45665789,-7.29448223,6.42922449,6.13602304,0.912955165,0.893933535,Mejores_poses/bases_conjuntas/EGFR_MAPK1_1000_...,Mejores_poses/bases_conjuntas/EGFR_MAPK1_1000_...,EGFR,MAPK1
...,...,...,...,...,...,...,...,...,...,...,...
1124,EGFR_MAPK1_999,-6.34970522,-7.63998604,6.34291601,7.43806696,0.382018745,0.676731169,Mejores_poses/bases_conjuntas/EGFR_MAPK1_999_E...,Mejores_poses/bases_conjuntas/EGFR_MAPK1_999_M...,EGFR,MAPK1
1125,EGFR_SRC_1147,-8.69953632,-8.68173599,7.57741594,7.48369837,0.475639999,0.78958863,Mejores_poses/bases_conjuntas/EGFR_SRC_1147_EG...,Mejores_poses/bases_conjuntas/EGFR_SRC_1147_SR...,EGFR,SRC
1126,EGFR_SRC_1148,-6.42355871,-5.79782248,6.44153261,6.74391508,0.47949934,0.764384985,Mejores_poses/bases_conjuntas/EGFR_SRC_1148_EG...,Mejores_poses/bases_conjuntas/EGFR_SRC_1148_SR...,EGFR,SRC
1127,EGFR_SRC_1149,-8.33077812,-8.07119846,7.15295744,7.44163275,0.469591349,0.50486213,Mejores_poses/bases_conjuntas/EGFR_SRC_1149_EG...,Mejores_poses/bases_conjuntas/EGFR_SRC_1149_SR...,EGFR,SRC


# Guardar datos de compuestos
<hr>

In [117]:
resultado.to_csv("Resultados_docking/Bases_conjuntas/Duales.csv")